## Selecting Rows with Core or ORM

For both Core and ORM, the `select()` function generates a `Select construct` which is used for all `SELECT` queries. Passed to methods like _`Connection.execute()` in Core_ and _`Session.execute()` in ORM_, a `SELECT statement` is emitted in the _current transaction_ and the result rows available via the returned `Result` object.

#### Initialize tables with some data

In [1]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String,
    ForeignKey, create_engine, insert, select, bindparam,
)
from sqlalchemy.orm import Session, registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-09-22 10:16:51,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:51,433 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-22 10:16:51,434 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,436 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-22 10:16:51,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,440 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-22 10:16:51,441 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,442 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-22 10:16:51,443 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,444 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-22 10:16:51,445 INFO sqlalchemy.engine.Engine [no key 0.00082s] ()
2022-09-22 10:16:51,447 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-09-22 10:16:51,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:51,527 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-22 10:16:51,529 INFO sqlalchemy.engine.Engine [generated in 0.00352s] ('spongebob', 'Spongebob Squarepants')
2022-09-22 10:16:51,533 INFO sqlalchemy.engine.Engine COMMIT
2022-09-22 10:16:51,535 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:51,536 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-22 10:16:51,537 INFO sqlalchemy.engine.Engine [generated in 0.00231s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-22 10:16:51,539 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-09-22 10:16:51,621 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:51,623 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-09-22 10:16:51,624 INFO sqlalchemy.engine.Engine [generated in 0.00230s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-09-22 10:16:51,625 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-09-22 10:16:51,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:51,896 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-22 10:16:51,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,900 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-22 10:16:51,901 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-22 10:16:51,902 INFO sqlalchemy.engine.Engine COMMIT


#### The `select()` SQL Expression Construct

The `select()` construct builds up a statement in the same way as that of `insert()`, using a _generative approach_ where each method builds more state onto the object. Like the other SQL constructs, it can be _stringified_ in place.

In [10]:
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Also in the same manner as all other statement-level `SQL constructs`, _to actually run the statement_ we pass it to an `execution` method. Since a `SELECT statement` returns _rows_ we can always iterate the result object to get _Row_ objects back.

In [11]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-09-22 10:16:52,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:52,055 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-22 10:16:52,056 INFO sqlalchemy.engine.Engine [generated in 0.00291s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-09-22 10:16:52,059 INFO sqlalchemy.engine.Engine ROLLBACK


When using the ORM, particularly with a `select()` construct that's _composed against ORM entities_, we will want to execute it using the `Session.execute()` method on the `Session`; using this approach, we continue to get `Row` objects from the result, however these rows are now _capable of including **complete entities**_, such as instances of the `User` class, as individual elements within each row.

In [12]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-09-22 10:16:52,132 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-22 10:16:52,140 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-22 10:16:52,141 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-09-22 10:16:52,144 INFO sqlalchemy.engine.Engine ROLLBACK


While the SQL generated in these examples looks the same whether we invoke `select(user_table)` or `select(User)`, in the more general case they _do not necessarily render the same thing_, as an ORM-mapped class may be mapped to other kinds of `"selectables"` besides tables. __The `select()` that's against an ORM entity also indicates that ORM-mapped instances should be returned in a result, which is not the case when SELECTing from a Table object.__